<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       DFFT and IDFFT Functions in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial'><b>DFFT and IDFFT</b></p>
<p style = 'font-size:16px;font-family:Arial'>The <b>DFFT()</b> function takes a time or space series as an input, and produces a result series containing the computed Fourier Transform Coefficients. The computed coefficients can be output in either rectangular (real, imaginary) or polar (amplitude, phase) forms.</p>

<p style = 'font-size:16px;font-family:Arial'>The <b>IDFFT()</b> function reverses the effects of the forward transform (DFFT() function). It takes a series containing Fourier Coefficients as an input, and returns the original series that was input into the DFFT() function to generate the coefficients. The Fourier Coefficients can be in either the rectangular (real, imaginary) or polar (amplitude, phase) formats.</p>

<p style = 'font-size:16px;font-family:Arial'>The following procedure is an example of how to use DFFT() when convolving two series with digital signal processing:</p>
<li style = 'font-size:16px;font-family:Arial'>Use DFFT() on series 1 and series 2 to get dataframes named 'dfftRes1' and 'dfftRes2', respectively.</li>
<li style = 'font-size:16px;font-family:Arial'>Use BinarySeriesOp() to do point-wise multiplication using 'dfftRes1' and 'dfftRes2'.</li>
<li style = 'font-size:16px;font-family:Arial'>Use IDFFT() on the output of BinarySeriesOp() to get the convolved result of the two series.</li></p>

</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    TDAnalyticResult,
    DFFT,
    IDFFT,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_DFFT_IDFFT_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", "mvdfft8")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data = DataFrame.from_table("mvdfft8")
data

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>3. DFFT and IDFFT</b>
<p style = 'font-size:16px;font-family:Arial'>The DFFT() is used on series 1 and series 2 to get dataframes named 'dfftRes1' and 'dfftRes2', respectively. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(DFFT)

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the DFFT function.</p>

In [ ]:
data_series_df = TDSeries(data=data,
                              id="sid",
                              row_index="n_seqno",
                              row_index_style="SEQUENCE",
                              payload_field="magnitude1",
                              payload_content="REAL")

In [ ]:
DFFT_result = DFFT(data=data_series_df,
                       human_readable=True,
                       output_fmt_content='COMPLEX')

dfft_df = DFFT_result.result
dfft_df

<p style = 'font-size:16px;font-family:Arial'>Compute the inverse fourier transform using TDAnalyticResult as input.</p>
<p style = 'font-size:16px;font-family:Arial'>Create teradataml TDAnalyticResult object to be passed as an input to IDFFT.</p>

In [ ]:
idfft_art_spec = TDAnalyticResult(data=DFFT_result.result,
                                      payload_content="COMPLEX",
                                      payload_field=["REAL_MAGNITUDE1",
                                                     "IMAG_MAGNITUDE1"])

<p style = 'font-size:16px;font-family:Arial'>Execute IDFFT function. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(IDFFT)

In [ ]:
uaf_out = IDFFT(data=idfft_art_spec, human_readable=True)
uaf_out.result

<p style = 'font-size:16px;font-family:Arial'>Compute the inverse fourier transform using TDSeries as input.</p>
<p style = 'font-size:16px;font-family:Arial'>Create a teradataml TDSeries object.</p>

In [ ]:
idfft_series_spec = TDSeries(data=DFFT_result.result,
                                 id="sid",
                                 row_index="sid",
                                 row_index_style="SEQUENCE",
                                 payload_content="COMPLEX",
                                 payload_field=["REAL_MAGNITUDE1",
                                                "IMAG_MAGNITUDE1"])

<p style = 'font-size:16px;font-family:Arial'>Execute IDFFT function</p>

In [ ]:
uaf_out = IDFFT(data=idfft_series_spec, human_readable=True)
uaf_out.result

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("mvdfft8")

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>DFFT function reference: <a href = 'https://docs.teradata.com/search/all?query=DFFT&content-lang=en-US'>here</a></li>
    <li>IDFFT function reference: <a href = 'https://docs.teradata.com/search/all?query=IDFFT&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>